# Quickstart for Pandas API on Spark

This is a short introduction to pandas API on Spark, geared mainly for new users. This notebook shows you some key differences between pandas and pandas API on Spark. You can run this examples by yourself on a live notebook [here](https://mybinder.org/v2/gh/pyspark.pandas/master?filepath=docs%2Fsource%2Fgetting_started%2Fquickstart_ps.ipynb).

Customarily, we import pandas API on Spark as follows:

In [1]:
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession

ModuleNotFoundError: No module named 'pyspark.pandas'

## Object Creation



Creating a pandas-on-Spark Series by passing a list of values, letting pandas API on Spark create a default integer index:

In [ ]:
s = ps.Series([1, 3, 5, np.nan, 6, 8])

In [ ]:
s

Creating a pandas-on-Spark DataFrame by passing a dict of objects that can be converted to series-like.

In [ ]:
psdf = ps.DataFrame(
    {'a': [1, 2, 3, 4, 5, 6],
     'b': [100, 200, 300, 400, 500, 600],
     'c': ["one", "two", "three", "four", "five", "six"]},
    index=[10, 20, 30, 40, 50, 60])

In [ ]:
psdf

Creating a pandas DataFrame by passing a numpy array, with a datetime index and labeled columns:

In [ ]:
dates = pd.date_range('20130101', periods=6)

In [ ]:
dates

In [ ]:
pdf = pd.DataFrame(np.random.randn(6, 4), index=dates, columns=list('ABCD'))

In [ ]:
pdf

Now, this pandas DataFrame can be converted to a pandas-on-Spark DataFrame

In [ ]:
psdf = ps.from_pandas(pdf)

In [ ]:
type(psdf)

It looks and behaves the same as a pandas DataFrame though

In [ ]:
psdf

Also, it is possible to create a pandas-on-Spark DataFrame from Spark DataFrame.  

Creating a Spark DataFrame from pandas DataFrame

In [ ]:
spark = SparkSession.builder.getOrCreate()

In [ ]:
sdf = spark.createDataFrame(pdf)

In [ ]:
sdf.show()

Creating pandas-on-Spark DataFrame from Spark DataFrame.
`to_koalas()` is automatically attached to Spark DataFrame and available as an API when pandas API on Spark is imported.

In [ ]:
psdf = sdf.to_pandas_on_spark()

In [ ]:
psdf

Having specific [dtypes](http://pandas.pydata.org/pandas-docs/stable/basics.html#basics-dtypes) . Types that are common to both Spark and pandas are currently supported.

In [ ]:
psdf.dtypes

## Viewing Data

See the [API Reference](https://koalas.readthedocs.io/en/latest/reference/index.html).

See the top rows of the frame. The results may not be the same as pandas though: unlike pandas, the data in a Spark dataframe is not _ordered_, it has no intrinsic notion of index. When asked for the head of a dataframe, Spark will just take the requested number of rows from a partition. Do not rely on it to return specific rows, use `.loc` or `iloc` instead.

In [ ]:
psdf.head()

Display the index, columns, and the underlying numpy data.

You can also retrieve the index; the index column can be ascribed to a DataFrame, see later

In [ ]:
psdf.index

In [ ]:
psdf.columns

In [ ]:
psdf.to_numpy()

Describe shows a quick statistic summary of your data

In [ ]:
psdf.describe()

Transposing your data

In [ ]:
psdf.T

Sorting by its index

In [ ]:
psdf.sort_index(ascending=False)

Sorting by value

In [ ]:
psdf.sort_values(by='B')

## Missing Data
Pandas API on Spark primarily uses the value `np.nan` to represent missing data. It is by default not included in computations. 


In [ ]:
pdf1 = pdf.reindex(index=dates[0:4], columns=list(pdf.columns) + ['E'])

In [ ]:
pdf1.loc[dates[0]:dates[1], 'E'] = 1

In [ ]:
psdf1 = ps.from_pandas(pdf1)

In [ ]:
psdf1

To drop any rows that have missing data.

In [ ]:
psdf1.dropna(how='any')

Filling missing data.

In [ ]:
psdf1.fillna(value=5)

## Operations

### Stats
Operations in general exclude missing data.

Performing a descriptive statistic:

In [ ]:
psdf.mean()

### Spark Configurations

Various configurations in PySpark could be applied internally in pandas API on Spark.
For example, you can enable Arrow optimization to hugely speed up internal pandas conversion. See <a href="https://spark.apache.org/docs/latest/sql-pyspark-pandas-with-arrow.html">PySpark Usage Guide for Pandas with Apache Arrow</a>.

In [ ]:
prev = spark.conf.get("spark.sql.execution.arrow.enabled")  # Keep its default value.
ps.set_option("compute.default_index_type", "distributed")  # Use default index prevent overhead.
import warnings
warnings.filterwarnings("ignore")  # Ignore warnings coming from Arrow optimizations.

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", True)
%timeit ps.range(300000).to_pandas()

In [ ]:
spark.conf.set("spark.sql.execution.arrow.enabled", False)
%timeit ps.range(300000).to_pandas()

In [ ]:
ps.reset_option("compute.default_index_type")
spark.conf.set("spark.sql.execution.arrow.enabled", prev)  # Set its default value back.

## Grouping
By “group by” we are referring to a process involving one or more of the following steps:

- Splitting the data into groups based on some criteria
- Applying a function to each group independently
- Combining the results into a data structure

In [ ]:
psdf = ps.DataFrame({'A': ['foo', 'bar', 'foo', 'bar',
                          'foo', 'bar', 'foo', 'foo'],
                    'B': ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                    'C': np.random.randn(8),
                    'D': np.random.randn(8)})

In [ ]:
psdf

Grouping and then applying the [sum()](https://koalas.readthedocs.io/en/latest/reference/api/pyspark.pandas.groupby.GroupBy.sum.html#databricks.koalas.groupby.GroupBy.sum) function to the resulting groups.

In [ ]:
psdf.groupby('A').sum()

Grouping by multiple columns forms a hierarchical index, and again we can apply the sum function.

In [ ]:
psdf.groupby(['A', 'B']).sum()

## Plotting
See the <a href="https://koalas.readthedocs.io/en/latest/reference/frame.html#plotting">Plotting</a> docs.

In [ ]:
pser = pd.Series(np.random.randn(1000),
                 index=pd.date_range('1/1/2000', periods=1000))

In [ ]:
psser = ps.Series(pser)

In [ ]:
psser = psser.cummax()

In [ ]:
psser.plot()

On a DataFrame, the <a href="https://koalas.readthedocs.io/en/latest/reference/api/pyspark.pandas.frame.DataFrame.plot.html#databricks.koalas.frame.DataFrame.plot">plot()</a> method is a convenience to plot all of the columns with labels:

In [ ]:
pdf = pd.DataFrame(np.random.randn(1000, 4), index=pser.index,
                   columns=['A', 'B', 'C', 'D'])

In [ ]:
psdf = ps.from_pandas(pdf)

In [ ]:
psdf = psdf.cummax()

In [ ]:
psdf.plot()

## Getting data in/out
See the <a href="https://koalas.readthedocs.io/en/latest/reference/io.html">Input/Output
</a> docs.

### CSV

CSV is straightforward and easy to use. See <a href="https://koalas.readthedocs.io/en/latest/reference/api/pyspark.pandas.DataFrame.to_csv.html#databricks.koalas.DataFrame.to_csv">here</a> to write a CSV file and <a href="https://koalas.readthedocs.io/en/latest/reference/api/databricks.koalas.read_csv.html#databricks.koalas.read_csv">here</a> to read a CSV file.

In [ ]:
psdf.to_csv('foo.csv')
ps.read_csv('foo.csv').head(10)

### Parquet

Parquet is an efficient and compact file format to read and write faster. See <a href="https://koalas.readthedocs.io/en/latest/reference/api/pyspark.pandas.DataFrame.to_parquet.html#databricks.koalas.DataFrame.to_parquet">here</a> to write a Parquet file and <a href="https://koalas.readthedocs.io/en/latest/reference/api/databricks.koalas.read_parquet.html#databricks.koalas.read_parquet">here</a> to read a Parquet file.

In [ ]:
psdf.to_parquet('bar.parquet')
ps.read_parquet('bar.parquet').head(10)

### Spark IO

In addition, pandas API on Spark fully supports Spark's various datasources such as ORC and an external datasource.  See <a href="https://koalas.readthedocs.io/en/latest/reference/api/pyspark.pandas.DataFrame.to_spark_io.html#databricks.koalas.DataFrame.to_spark_io">here</a> to write it to the specified datasource and <a href="https://koalas.readthedocs.io/en/latest/reference/api/databricks.koalas.read_spark_io.html#databricks.koalas.read_spark_io">here</a> to read it from the datasource.

In [ ]:
psdf.to_spark_io('zoo.orc', format="orc")
ps.read_spark_io('zoo.orc', format="orc").head(10)